Assignment 2 - Regression and Classification Models

1. Load the libraries - all in one for the whole assignment
2. Load the data
3. Inspect and clean the csv - I used the same data_preprocessor.py file from the previous assignment

I googled A LOT, I used examples of functions that I found online (I added the links). ChatGPT was my troubleshooting companion. 

In [ ]:
### loading libraries ###
import data_preprocessor as dp
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_curve, auc, precision_recall_curve, average_precision_score
import pandas as pd

### loading data ###
messy_data = pd.read_csv(r'../Data/heart_disease_uci.csv', na_values=['', 'NA', 'Na', 'null', ""])
working_data = messy_data.copy()

Look into the data - used the data wrangler to get a better understanding of the data.
I basically did the steps similar to the first assignmnent, using the already defined functions in data_preprocessor.py

Steps:
- drop duplicates
- remove columns and rows missing > 25% data
- as "chol" and "num" columns are targers, I deleted rows with missing values in "chol" ("num" didn;t have missing data)
- I visualized outliers in numeric columns and then deleted them
- I dropped 9 more rows instead of imputing. 1 from "restecg" and 8 from "fbs". Normally I would impute the values but both of those are cathegorical columns meaning imputing would either hit the target or not and the overall number of rows was very low.
- I also looked for correlation but didn't find it.

Please feel free to un-comment portions of code to double check my work.

In [ ]:
### cleaning the data ###
working_data = working_data.drop_duplicates()
working_data = dp.remove_columns_missing_data(working_data)
working_data = dp.remove_rows_missing_data(working_data)
# target columns: "chol" and "num"
working_data = working_data.dropna(subset = "chol")

# for column in ["trestbps","chol","thalch","oldpeak"]:
#     sns.boxplot(x=working_data[column])
#     plt.title(f"Boxplot of {column}")
#     plt.show()

columns = ["trestbps","chol","thalch","oldpeak"]
working_data = dp.remove_outliers(working_data,columns)

working_data = working_data.dropna(subset = "fbs") # just 8
working_data = working_data.dropna(subset = "restecg") # just 1

# print(working_data[["trestbps","chol","thalch","oldpeak","num"]].corr())
# no correlation found

clean_data = working_data
working_data.to_csv(r"../Data/clean_data.csv", index=False)


1. Regression model

The next stop was splitting the data for training and testing at the standard 80-20. In the first model we're looking to predict cholesterol, y, based on everything else, x.
Steps:
- assign x and y values
- convert categotical data into numeric. this is what google suggested cause apparently "male" is not a scalar
- split data

Standardizing data - normalize=True parameter didn't work. I used StandardScaler. 
(source: https://dataaspirant.com/elasticnet-regression-python/)

In [ ]:
### split data into training and test ###
# define x and y 
X = clean_data.drop(columns='chol')
y = clean_data['chol']

# convert categorical columns to numeric
X = pd.get_dummies(X, drop_first=True)

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize data
scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.fit_transform(X_test)

Train the model - 
I looped the model through the pre-chosen alpha and l1_ratio values. I created the results dataframe to see what's going on - i needed this dataframe for chosing the best configuration.
Then I chose to use hyperparameter grid to determine the best parameters.
(source: https://dataaspirant.com/elasticnet-regression-python/)

Choice of l1 rations: 0.1, 0.5, 0.9: almost entirely L1, evenly split, almost entirely L2 and alphas: I chose logarythmically spaced values to have the most information with the least amount of values.

Print output:
Best alpha: 1.0
Best l1_ratio: 0.1

In [ ]:
# Training the model
alphas = [0.1, 1, 10]
l1_ratios = [0.1, 0.5, 0.9]
results=[]

for alpha in alphas:
    for l1_ratio in l1_ratios:
        model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        model.fit(scaled_x_train, y_train)
        y_pred = model.predict(scaled_x_test)
        rmse = mean_squared_error(y_test, y_pred, squared=False)
        r2 = r2_score(y_test, y_pred)
        results.append((alpha, l1_ratio, rmse, r2))

results_df = pd.DataFrame(results, columns=["alpha", "l1_ratio", "RMSE", "R2"])
results_df.to_csv(r"../Data/results_df.csv", index=False)

# Using hyperparameter grid to determine best alpha and l1 ratio
param_grid = {
    'alpha': [0.1, 1, 10],
    'l1_ratio': [0.1, 0.5, 0.9],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid)
grid_search.fit(scaled_x_train, y_train)

print('Best alpha:', grid_search.best_estimator_.alpha)
print('Best l1_ratio:', grid_search.best_estimator_.l1_ratio)

Determining the optimal configuration. 

I'm leaving this part for your enjoyment. After 2h of trying to reinvent the wheel, I realised GridSearchCV actually gave me the answer to this question. 

Steps:
- minmax normalize R2 and RMSE (used the function from data_preprocessor.py)
- because higher R2 is better and lower RMSE is better, I made a column with 1-RMSE to invert the column. (this way max sum would be 2)
- sum up R2 and 1-RMSE
- I stopped the insanity here and just read the highest value from the csv.

best parameters (config):
alpha: 1.0
l1 ratio: 0.1

In [ ]:

results_df["R2_scaled"] = dp.normalize_data(results_df[["R2"]].copy(), ["R2"])["R2"]
# results_df.to_csv(r"../Data/results_df.csv", index=False)

results_df["RMSE_scaled"] = dp.normalize_data(results_df[["RMSE"]].copy(), ["RMSE"])["RMSE"]
results_df["RMSE_scaled_reversed"] = 1- dp.normalize_data(results_df[["RMSE"]].copy(), ["RMSE"])["RMSE"]

results_df["final_score"] = results_df["R2_scaled"] + results_df["RMSE_scaled_reversed"]

results_df.to_csv(r"../Data/results_df.csv", index=False)


Visualization!
Heatmaps are cool.

In [ ]:
# Plot R2 heatmap
r2_pivot = results_df.pivot(index="alpha", columns="l1_ratio", values="R2")
sns.heatmap(r2_pivot, annot=True)
plt.title("R² Heatmap (ElasticNet)")
plt.xlabel("l1_ratio")
plt.ylabel("alpha")
plt.show()

# Plot RMSE heatmap
rmse_pivot = results_df.pivot(index="alpha", columns="l1_ratio", values="RMSE")
sns.heatmap(rmse_pivot, annot=True)
plt.title("RMSE Heatmap (ElasticNet)")
plt.xlabel("l1_ratio")
plt.ylabel("alpha")
plt.show()


2. Classification Models

I used the same clean_data, divided and standardized features the same way as above. Only this time the target (y) was predicting the presence or absence of heart disease (column "num"). The other difference is because having a disease is a 0 or 1 kind of thing, I binarized the outcome. 

In [ ]:
### split data into training and test ###
# define x and y 
X = clean_data.drop(columns='num')
y = clean_data['num']
X = pd.get_dummies(X, drop_first=True)

# binarize the outcome
y = (y > 0).astype(int) 

# split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# standardize data
scaler = StandardScaler()
scaled_x_train = scaler.fit_transform(X_train)
scaled_x_test = scaler.fit_transform(X_test)

Train logistic regression model
(source: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html mixed with lab3) 
I used the list of solvers and penalties from the link above too. Not all are compatible so I used try-except while looping through the possibilities.
Then I summed up the evaluation metrics to fund the best one. 
I identified the best combo by opening the csv.

The best combo: saga, l1
2nd best: liblinear,l1
Note: accuracy and f1 were the same for all combinations, the rest had very minor differences.

The best scores are to save the data of the best config to make the figure.

If I did it again, I'd put the figure function under the loop to have more figures/data to compare, not just the figure of the best one. That goes for kNN too.

In [ ]:
### LogisticRegression
solvers = ["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag", "saga"]
penalties = ['l1', 'l2']
results = []
best_score = 0
best_log_model = None
log_best_fpr = log_best_tpr = log_best_precision = log_best_recall = None

for solver in solvers:
    for penalty in penalties:
        try:
            model = LogisticRegression(solver=solver, penalty=penalty, max_iter=10000)
            model.fit(X_train_scaled, y_train)
            y_scores = model.predict_proba(X_test_scaled)[:, 1]
            y_pred = model.predict(X_test_scaled)

            # evaluation metrics:
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            fpr, tpr, _ = roc_curve(y_test, y_scores)
            auroc = auc(fpr, tpr) 
            precision, recall, _ = precision_recall_curve(y_test, y_scores)
            auprc = average_precision_score(y_test, y_scores)

            score = accuracy + f1 + auroc + auprc
            if score > best_score:
                best_score = score
                best_log_model = model
                log_best_fpr = fpr
                log_best_tpr = tpr
                log_best_precision = precision
                log_best_recall = recall

            #print(f"pair: {solver} - {penalty}")
            results.append((solver, penalty, accuracy, f1, auroc, auprc, fpr, tpr, precision, recall))
        except Exception as e:
            #print(f"Skipped: solver={solver}, penalty={penalty} — {e}")
            continue

results_summary = pd.DataFrame(results, columns=["solver", "penalty", "accuracy", "f1", "auroc", "auprc", "_", "_", "_", "_"])
results_summary = results_summary.drop(columns=["_", "_", "_", "_"])
# results_summary.to_csv(r"../Data/results_summary.csv", index=False)

# find the best config:
results_summary["sum"] = results_summary[["accuracy", "f1", "auroc", "auprc"]].sum(axis=1)
results_summary.to_csv(r"../Data/results_summary.csv", index=False)


k-NN
I looped the k-NN over the values mentioned in the assignment [1,5,10]
For the code, I adjusted the code from lab4.
Overall the idea was the same as above - looping throgh the k values and saving 

In [ ]:
k_values = [1, 5, 10]
best_knn_score = 0
best_knn_model = None
knn_best_fpr = knn_best_tpr = knn_best_precision = knn_best_recall = None

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k, metric='manhattan')
    knn.fit(X_train_scaled, y_train)
    y_scores_knn = knn.predict_proba(X_test_scaled)[:, 1]
    y_pred_knn = knn.predict(X_test_scaled)

    accuracy = accuracy_score(y_test, y_pred_knn)
    f1 = f1_score(y_test, y_pred_knn)
    fpr, tpr, _ = roc_curve(y_test, y_scores_knn)
    auroc = auc(fpr, tpr)
    precision, recall, _ = precision_recall_curve(y_test, y_scores_knn)
    auprc = average_precision_score(y_test, y_scores_knn)

    score = accuracy + f1 + auroc + auprc
    if score > best_knn_score:
        best_knn_score = score
        best_knn_model = knn
        knn_best_fpr, knn_best_tpr = fpr, tpr
        knn_best_precision, knn_best_recall = precision, recall
    
    knn_results.append((k, accuracy, f1, auroc, auprc))

knn_results_summary = pd.DataFrame(knn_results, columns=["k", "accuracy", "f1", "auroc", "auprc"])
knn_results_summary["sum"] = knn_results_summary[["accuracy", "f1", "auroc", "auprc"]].sum(axis=1)
knn_results_summary.to_csv(r"../Data/knn_results_summary.csv", index=False)



All the plots.
The function for the plots was also taken from lab4 and is defined in data_preprocessor.py file.
First regression, then k-NN
Both plots were used for the best parameters.

In [ ]:
# Best logistic regression model curves
log_auprc = average_precision_score(y_test, best_log_model.predict_proba(X_test_scaled)[:, 1])
dp.plot_curves(log_best_tpr, log_best_fpr, auc(log_best_fpr, log_best_tpr), log_best_precision, log_best_recall, log_auprc, 'Logistic Regression', minority_class=y.mean())

# Best k-NN model curves
knn_auprc = average_precision_score(y_test, best_knn_model.predict_proba(X_test_scaled)[:, 1])
dp.plot_curves(knn_best_tpr, knn_best_fpr, auc(knn_best_fpr, knn_best_tpr), knn_best_precision, knn_best_recall, knn_auprc, 'k-NN', minority_class=y.mean())
